In [1]:
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
import warnings
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.utils.multiclass import unique_labels

warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('ds_challenge_data_final.csv',sep='\t')
#df.fillna('none', inplace = True)
df.fillna(0, inplace = True)
#df = df.sample(frac=0.35)
df.columns.to_list()

In [3]:
pred_cat_cols = ['holiday_nm']
cols_to_drop = ['Transaction_Date','Hash_Trans_ID','PRO' ]

In [4]:
df = pd.get_dummies(data=df, columns = pred_cat_cols)

In [5]:
df_train = df[df['Transaction_Year'] < 2015]
train_targets = df_train['PRO'].as_matrix().astype('float32')
train_data = df_train.drop(cols_to_drop, axis=1, inplace=False).as_matrix().astype('float32')

In [6]:
df_test = df[df['Transaction_Year'] == 2015]
test_targets = df_test['PRO'].as_matrix().astype('float32')
test_data = df_test.drop(cols_to_drop, axis=1, inplace=False).as_matrix().astype('float32')

In [7]:
mdl = xgb.XGBClassifier(max_depth=2, learning_rate=0.2, n_estimators=100, verbosity=1, n_jobs=-1)
#mdl = RandomForestClassifier(n_estimators=100, min_samples_leaf=50, n_jobs=-1, max_depth=20)
#mdl = ExtraTreesClassifier(n_estimators=100, min_samples_leaf=5, n_jobs=-1, max_depth=20)
#mdl = GradientBoostingClassifier(n_estimators=100, min_samples_leaf=5, max_depth=20)

In [8]:
mdl.fit(train_data,train_targets)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bytree=1, gamma=0, learning_rate=0.2, max_delta_step=0,
              max_depth=2, min_child_weight=1, missing=None, n_estimators=100,
              n_jobs=-1, nthread=None, objective='binary:logistic',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              seed=None, silent=True, subsample=1, verbosity=1)

In [9]:
test_pred = mdl.predict_proba(test_data)

In [10]:
predictions = np.array(test_pred)[:,1]
actuals = test_targets.astype('int')

In [11]:
fpr, tpr, thresholds = metrics.roc_curve(actuals, predictions, pos_label=1)
metrics.auc(fpr, tpr)

0.7121332659432172